## Data statistics for the population and each cluster

In [1]:
# Include and load packages, config files
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from utils.data_utils import load_all_data, load_covariates
from utils.utils import compute_simlr, feat_ranking, estimate_number_clusters
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Parameters of the procedure
clusters = 4
rd_seed = 1714                                          # Random seed for experiment replication

# Paths
existing_cluster = True                               # Compute the clustering again or use an existing one
cluster_path = "/home/gerard/Documents/EXPERIMENTS/SIMLR-AD/cimlr4long/"# Path of the existing cluster, if applicable
covariate_path = "../long_data/covariates_long.csv"       # Path of the covariance data frame (.csv)

#covariate_data, cov_names, feature_data, feature_names = load_all_data(covariate_path, feature_path)
#covariate_data_nonorm, cov_names, feature_data_nonorm, feature_names = load_all_data(covariate_path, feature_path, normalize=False)
#feature_data['DX'] = covariate_data.DX_bl.values

covariate_data, cov_names = load_covariates(covariate_path, normalize=True)
covariate_data_nonorm, cov_names = load_covariates(covariate_path, normalize=True)


if existing_cluster:
    # Load existent
    c_data = pd.read_csv(cluster_path + 'cluster_data.csv')
    ## Load S, F data
    S = np.load(cluster_path + 'S_matrix.npy')
    ydata = np.load(cluster_path + 'ydata_matrix.npy')
    F = np.load(cluster_path + 'F_matrix.npy')
else:
    # Compute base clustering
    y_b, S, F, ydata, alpha = compute_simlr(
        np.array(covariate_data[cov_names]), clusters)

#names = pd.DataFrame(data={
#    "cov_names": cov_names})
#names.to_csv(cluster_path + 'cov_names.csv')

#names = pd.DataFrame(data={
#    "feat_names": feature_names})
#names.to_csv(cluster_path + 'feat_names.csv')

#ptid_latex = [x for x in map(lambda x: '$' + x +'$', covariate_data.PTID.values)]
#names = pd.DataFrame(data={
#    "id": ptid_latex})
#names.to_csv(cluster_path + 'ptid.csv')

Compute the basic summary for each of the diagnostic groups

In [2]:
# Check distribution of DX in each cluster
unique, counts = np.unique(covariate_data.DX_bl.values, return_counts=True)
print(dict(zip(unique, counts)))
print(len(covariate_data))
labels = ["MMSE", "PTEDUCAT", "AGE"]
otherlabels = ["PTGENDER", "APOE4"]
dx = ["CN", "LMCI", "AD"]
print(covariate_data[labels].describe())

g_ratio = len(covariate_data[covariate_data.PTGENDER.values == 0]) / len(covariate_data)
print(g_ratio)
# Apoe positive ratio
a_ratio = len(covariate_data[covariate_data.APOE4.values > 0]) / len(covariate_data)
print(a_ratio)

# For different diagnostics
for d in dx:
    print(d)
    c1 = covariate_data[covariate_data['DX_bl'].values == d]
    # Gender ratio (% of females)
    g_ratio = len(c1[c1.PTGENDER.values == 0]) / len(c1)
    print(g_ratio)
    # Apoe positive ratio
    a_ratio = len(c1[c1.APOE4.values > 0]) / len(c1)
    print(a_ratio)
    
    print(c1[labels].describe())


{'AD': 83, 'CN': 52, 'LMCI': 300}
435
             MMSE    PTEDUCAT         AGE
count  435.000000  435.000000  435.000000
mean    26.590805   15.600000   74.793333
std      2.380394    2.937286    7.320836
min     20.000000    6.000000   55.100000
25%     25.000000   14.000000   70.500000
50%     27.000000   16.000000   75.300000
75%     29.000000   18.000000   80.250000
max     30.000000   20.000000   89.600000
0.3931034482758621
0.5241379310344828
CN
0.4807692307692308
0.09615384615384616
            MMSE   PTEDUCAT        AGE
count  52.000000  52.000000  52.000000
mean   28.942308  15.615385  75.219231
std     1.178458   2.891189   5.848246
min    25.000000   8.000000  62.000000
25%    29.000000  13.000000  71.325000
50%    29.000000  16.000000  73.200000
75%    30.000000  18.000000  79.075000
max    30.000000  20.000000  89.600000
LMCI
0.36333333333333334
0.5566666666666666
             MMSE    PTEDUCAT         AGE
count  300.000000  300.000000  300.000000
mean    27.033333   15.69

In [3]:
# Check distribution of DX in each cluster
# Create dataframe of results
df_results = pd.DataFrame(index=["Nº subj","CN","MCI","AD","Age","Gender","Education","APoe4", "MMSE"])

unique, counts = np.unique(covariate_data.DX_bl.values, return_counts=True)

labels = ["MMSE", "PTEDUCAT", "AGE"]
otherlabels = ["PTGENDER", "APOE4"]
dx = ["CN", "LMCI", "AD"]

statistics = covariate_data[labels].describe()
print(statistics)
g_ratio = int((len(covariate_data[covariate_data.PTGENDER.values == 0]) / len(covariate_data)) * 100)

# Apoe positive ratio
a_ratio = int((len(covariate_data[covariate_data.APOE4.values > 0]) / len(covariate_data)) * 100)

data_cluster = [sum(counts), counts[1],counts[2],counts[0],"{:.2f}".format(statistics.loc["mean","AGE"]) + '+-' + "{:.2f}".format(statistics.loc["std","AGE"]), 
                str(g_ratio) + '%' , "{:.2f}".format(statistics.loc["mean","PTEDUCAT"]) + '+-' + "{:.2f}".format(statistics.loc["std","PTEDUCAT"]),
                str(a_ratio) + '%' , "{:.2f}".format(statistics.loc["mean","MMSE"]) + '+-' + "{:.2f}".format(statistics.loc["std","MMSE"])]
df_results["Total"] = pd.Series(data_cluster, index=df_results.index)


# For different clusters
for c in range(1, clusters+1):
    print('Cluster ' + str(c))
    c1 = covariate_data[c_data['C'].values == c]

    unique, counts = np.unique(c1.DX_bl.values, return_counts=True)

    # Gender ratio (% of females)
    g_ratio = int((len(c1[c1.PTGENDER.values == 0]) / len(c1)) * 100)

    # Apoe positive ratio
    a_ratio = int((len(c1[c1.APOE4.values > 0]) / len(c1)) * 100)

    statistics = c1[labels].describe()
    data_cluster = [sum(counts), counts[1],counts[2],counts[0],"{:.2f}".format(statistics.loc["mean","AGE"]) + '+-' + "{:.2f}".format(statistics.loc["std","AGE"]), 
                    str(g_ratio) + '%' , "{:.2f}".format(statistics.loc["mean","PTEDUCAT"]) + '+-' + "{:.2f}".format(statistics.loc["std","PTEDUCAT"]),
                    str(a_ratio) + '%' , "{:.2f}".format(statistics.loc["mean","MMSE"]) + '+-' + "{:.2f}".format(statistics.loc["std","MMSE"])]
    df_results[c] = pd.Series(data_cluster, index=df_results.index)

df_results.to_csv(cluster_path+'cluster_stats.csv')    

             MMSE    PTEDUCAT         AGE
count  435.000000  435.000000  435.000000
mean    26.590805   15.600000   74.793333
std      2.380394    2.937286    7.320836
min     20.000000    6.000000   55.100000
25%     25.000000   14.000000   70.500000
50%     27.000000   16.000000   75.300000
75%     29.000000   18.000000   80.250000
max     30.000000   20.000000   89.600000
Cluster 1
Cluster 2
Cluster 3
Cluster 4


In [ ]:
# For clusters 1 check the statistic for each diagnostic groups
c = 1
print('Cluster ' + str(c))
c1 = covariate_data[c_data['C'].values == c]
print(len(c1))
for d in dx:
    print(d)
    c2 = c1[c1['DX_bl'].values == d]
    # Gender ratio (% of females)
    g_ratio = len(c2[c2.PTGENDER.values == 0]) / len(c2)
    print("Gender:" + str(g_ratio))
    # Apoe positive ratio
    a_ratio = len(c2[c2.APOE4.values > 0]) / len(c2)
    print(a_ratio)
    print(c2[labels].describe())


In [ ]:
# Data statistics for important features in feature space

# import wilkcoxon test
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu

cluster_pairs = [(1,2), (1,3), (2,3)]

for c in cluster_pairs:
    print(c)
    c1 = covariate_data[c_data['C'].values == c[0]]
    c2 = covariate_data[c_data['C'].values == c[1]]
    for f in important_names:
        u, p_value = mannwhitneyu(c1[f].values, c2[f].values)
        print(f + ': ' + '{:.2}'.format(p_value))


In [ ]:
# Data statistics for important features in feature space

# import wilkcoxon test
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu

cluster_pairs = [(1,2), (1,3), (2,3)]
diagnostic = ["CN", "LMCI", "AD"]

for c in cluster_pairs:
    print(c)
    c1 = feature_data[c_data['C'].values == c[0]]
    c2 = feature_data[c_data['C'].values == c[1]]
    for d in diagnostic:
        print(d)
        d1 = c1[c1.DX.values == d]
        d2 = c2[c2.DX.values == d]
        for f in feature_names:
            u, p_value = mannwhitneyu(d1[f].values, d2[f].values)
            print(f + ': ' + '{:.2}'.format(p_value))
